In [1]:
# Reference : https://github.com/Hvass-Labs/TensorFlow-Tutorials

#import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import pickle
import prettytensor as pt
import time

%matplotlib inline

In [2]:
# Load CIFAR10
cifar_dir = "./data/CIFAR-10/cifar-10-batches-py/"
cifar_train_cache = "./data/CIFAR-10/inception_cifar10_train.pkl"
cifar_test_cache = "./data/CIFAR-10/inception_cifar10_test.pkl"
train_sz = 50000
test_sz = 10000
img_sz = 32
num_channels = 3
num_classes = 10


In [3]:
def unpickle(filename):
    fpath = cifar_dir+filename
    print("Loading "+fpath)
    file = open(fpath,mode='rb')
    data = pickle.load(file, encoding='bytes')
#     data = pickle.load(file, encoding='utf-8')
    file.close()
    return data

def decode(filename):
    data = unpickle(filename)
    raw_images = data[b'data']
    cls = np.array(data[b'labels'])
    images = np.array(raw_images, dtype=float).reshape([-1, num_channels, img_sz, img_sz])
    images = images.transpose([0, 2, 3, 1])
    return images, cls

def one_hotit(data):
    return np.eye(num_classes, dtype=float)[data]

def load_data():
    print("Loading tarin data")
    train_images = np.zeros(shape=[train_sz, img_sz, img_sz, num_channels], dtype=float)
    train_int_labels = np.zeros(shape=[train_sz], dtype=int)

    begin = 0
    for i in range(5):
        data_file = "data_batch_"+str(i+1)
        print("Loading "+data_file)
        images_batch, cls_batch = decode(data_file)
        num_images = len(images_batch)
        end = begin + num_images
        train_images[begin:end, :] = images_batch
        train_int_labels[begin:end] = cls_batch
        begin = end
        
    print("Loading test data")
    test_images, test_int_labels = decode("test_batch")
    return train_images, train_int_labels, test_images, test_int_labels



In [4]:
train_images, train_int_labels, test_images, test_int_labels = load_data()

Loading tarin data
Loading data_batch_1
Loading ./data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data_batch_2
Loading ./data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data_batch_3
Loading ./data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data_batch_4
Loading ./data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data_batch_5
Loading ./data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading test data
Loading ./data/CIFAR-10/cifar-10-batches-py/test_batch


In [5]:
train_labels = one_hotit(train_int_labels)
test_labels = one_hotit(test_int_labels)

In [6]:
metadata = unpickle("batches.meta")[b'label_names']
classnames = [x.decode('utf-8') for x in metadata]

Loading ./data/CIFAR-10/cifar-10-batches-py/batches.meta


In [8]:
# Load inception model
# Reference : https://github.com/Hvass-Labs/TensorFlow-Tutorials
import inception
from inception import transfer_values_cache
inception.maybe_download()
model = inception.Inception()

- Download progress: 100.0%
Download finished. Extracting files.
Done.


In [ ]:
train_transfer_values = transfer_values_cache(cache_path=cifar_train_cache , images=train_images, model=model)

In [ ]:
test_transfer_values = transfer_values_cache(cache_path=cifar_test_cache , images=test_images, model=model)

In [ ]:
# TF variables
x = tf.placeholder(tf.float32, shape=[None, model.transfer_len], name='x')
y_onehot = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_onehot')
y_int = tf.argmax(y_onehot, dimension=1)


# Neural Network
x_pretty = pt.wrap(x)
with pt.defaults_scope(activation_fn=tf.nn.relu):
    y_pred, loss = x_pretty.\
    fully_connected(size=100, name='layer_fc1').\
    fully_connected(size=10, name='layer_fc2').\
    softmax_classifier(num_classes=num_classes, labels=y_onehot)

# Optimizer and step counter
global_step = tf.Variable(initial_value=0, name='global_step', trainable=False)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss, global_step)
    
# TF test 
ty_int = tf.argmax(y_pred, dimension=1)
correct_prediction = tf.equal(y_int, ty_int)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
session = tf.Session()

In [ ]:
session.run(tf.global_variables_initializer())

In [ ]:
batch_size = 256 # Change to limit RAM consumption

In [ ]:
def random_batch():
    idx = np.random.choice(train_sz, size=batch_size, replace=False)
    x_batch = train_transfer_values[idx]
    y_batch = train_labels[idx]
    return x_batch, y_batch

In [ ]:
def optimize(num_iterations):
    tstart = time.time()
    for i in range(num_iterations):
        x_batch, y_batch = random_batch()
        feed_dict_train = {x: x_batch, y_onehot: y_batch}
        i_global,_ = session.run([global_step, optimizer], feed_dict=feed_dict_train)
        if (i_global % 50 == 0) or (i == num_iterations - 1):
            batch_acc = session.run(accuracy, feed_dict=feed_dict_train)
            msg = "Global Step: {0:>6}, Training Batch Accuracy: {1:>6.1%}"
            print(msg.format(i_global, batch_acc))

    tend = time.time()
    diff = tend - tstart
    print("Total time : " + str(diff) + " seconds")

In [ ]:
def predict():
    pred = np.zeros(shape=test_sz, dtype=np.int)
    i = 0
    while i < test_sz:
        j = min(i + batch_size, test_sz)
        feed_dict = {x: test_transfer_values[i:j], y_onehot: test_labels[i:j]}
        pred[i:j] = session.run(ty_int, feed_dict=feed_dict)
        i = j
    correct = (test_int_labels == pred)
    return correct, pred

In [ ]:
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(cls_pred):
    cm = confusion_matrix(y_true=test_int_labels, y_pred=cls_pred)
    for i in range(num_classes):
        # Append the class-name to each line.
        classname = "({}) {}".format(i, classnames[i])
        print(cm[i, :], classname)
    # Print the class-numbers for easy reference.
    class_numbers = [" ({0})".format(i) for i in range(num_classes)]
    print("".join(class_numbers))

In [ ]:
def print_accuracy():
    correct, pred = predict()
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(correct.mean(), correct.sum(), test_sz))
    print("Confusion Matrix:")
    plot_confusion_matrix(pred)

In [ ]:
print_accuracy()

In [ ]:
optimize(num_iterations=50)

In [ ]:
print_accuracy()

In [ ]:
import sys
sys.path.insert(0, '../')
from library.plot_tools import plot_tools
import sklearn.metrics as skm

correct, pred = predict()
plot_tools.plot_confusion_matrix(test_int_labels, pred, classes=classnames,
                                 normalize=True, title='Confusion matrix for test set using Inception model')